In [ ]:
#frame kodu
import cv2
import os
import json
import unicodedata

# Türkçe karakterleri ASCII uyumlu hale getir
def normalize_filename(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

def process_video(video_path, label, output_folder, json_data):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Video açılırken hata oluştu: {video_path}")
        return
    
    frame_count = 0
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = f"{label}_{frame_count}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)

        try:
            cv2.imwrite(frame_path, frame)
            print(f"Görsel kaydedildi: {frame_path}")
        except Exception as e:
            print(f"Resim kaydedilirken hata oluştu: {e}")
            continue

        frames.append({
            "image_path": frame_path,
            "label": label
        })

        frame_count += 1

    cap.release()
    json_data[label] = frames
    print(f"{label} etiketli video işlendi, {frame_count} kare kaydedildi.")

def save_json(json_data, output_json_path):
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"Veriler {output_json_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"JSON dosyasına yazılırken hata oluştu: {e}")

def process_videos(video_folder, output_folder, json_output_path):
    json_data = {}

    for video_name in os.listdir(video_folder):
        if video_name.endswith(".mp4"):
            label = os.path.splitext(video_name)[0]
            label_normalized = normalize_filename(label)  # Türkçe karakterleri sadeleştir

            video_path = os.path.join(video_folder, video_name)
            print(f"{video_name} işleniyor... (Etiket: {label_normalized})")
            process_video(video_path, label_normalized, output_folder, json_data)

    save_json(json_data, json_output_path)

# Ana çağrı
if __name__ == "__main__":
    video_folder = r"İşaretDiliVideo"
    output_folder = r"frameImages"
    json_output_path = r"frame_data.json"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    process_videos(video_folder, output_folder, json_output_path)
